
<a id="1"></a> <br>
## 2. Loading the data

In [17]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import metrics
import time, datetime
from sklearn import *
pd.set_option('display.max_columns', 500)

In [18]:
import feather
#test =feather.read_dataframe('tmp/microsoftv3_4_test')

#train =feather.read_dataframe('tmp/microsoft_train_prep_v1_2m')
#train =feather.read_dataframe('tmp/microsoft_train_prep_v2_2m')
                              #tmp/microsoft_train_prep_v1_full')
train =feather.read_dataframe('tmp/microsoft_train_prep_v2_fullb')
    #'tmp/trainwithdata')

In [19]:

#test =feather.read_dataframe('tmp/microsoftv3_train')#
#test =feather.read_dataframe('tmp/microsoft_test_prep_v2_2m')
    #'tmp/microsoft_test_prep_v1_full')  #'tmp/microsoftv3_train'
#train.info()
test =feather.read_dataframe('tmp/microsoft_test_prep_v2_fullb')

In [20]:
test.shape

(7853253, 90)

In [21]:
train.shape

(8921482, 90)

In [22]:
target = train['HasDetections']
del train['HasDetections']

In [23]:
train['third']=train['third'].astype('category')
train['fourth']=train['fourth'].astype('category')
test['third']=test['third'].astype('category')
test['fourth']=test['fourth'].astype('category')

In [24]:
remove_variables=['Census_ActivationChannel',
 'Census_FlightRing',
#                  'CityIdentifier',
 'Census_OSEdition',
 'Census_PowerPlatformRoleName',
 'Census_GenuineStateName',
 'Census_OSSkuName',
 'Census_OSBranch',
# 'Census_OSVersion',
 'Census_InternalBatteryType',
 'ProductName',
 'Census_ChassisTypeName',
 'Census_OSArchitecture',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName']


In [12]:
train.columns
to be tested:
param = {'numleaves': 200, 'mindatainleaf': 60, 
'objective':'binary',
'maxdepth': -1, 'learningrate': 0.1,
"boosting": "gbdt",
"featurefraction": 0.8, "baggingfreq": 1,
"baggingfraction": 0.8, "baggingseed": 11,
"metric": 'auc',
"lambdal1": 0.1, "randomstate": 7,
"verbosity": -1,
"n_jobs":-1}

Index(['ProductName', 'EngineVersion', 'AppVersion', 'AvSigVersion', 'IsBeta',
       'RtpStateBitfield', 'IsSxsPassiveMode', 'DefaultBrowsersIdentifier',
       'AVProductStatesIdentifier', 'AVProductsInstalled', 'AVProductsEnabled',
       'HasTpm', 'CountryIdentifier', 'CityIdentifier',
       'OrganizationIdentifier', 'GeoNameIdentifier',
       'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsVer',
       'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'OsBuildLab',
       'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode', 'SMode',
       'IeVerIdentifier', 'SmartScreen', 'Firewall', 'UacLuaenable',
       'Census_MDC2FormFactor', 'Census_DeviceFamily',
       'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_ProcessorClass',
       'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName',
       'Census_SystemVolumeTotalCapa

In [25]:
print(len(train.columns))
train=train.drop(remove_variables, axis=1)
print(len(train.columns))

89
76


In [26]:
print(len(test.columns))
test=test.drop(remove_variables, axis=1)
print(len(test.columns))

90
77


In [ ]:
for sub sampling
print(train.shape)
train=train.sample(frac=0.2, replace=False)
print(train.shape)

<a id="3"></a> <br>
## 3. Training the model

We set the max number of iteration over folds:

In [15]:

train.shape

(8921482, 74)

In [27]:

param = {'numleaves': 200, 'mindatainleaf': 60, 
'objective':'binary',
'maxdepth': -1, 'learningrate': 0.1,
"boosting": "gbdt",
"featurefraction": 0.8, "baggingfreq": 1,
"baggingfraction": 0.8, "baggingseed": 11,
"metric": 'auc',
"lambdal1": 0.1, "randomstate": 7,
"verbosity": -1,
"n_jobs":-1}

auc=list()
for i in [1]:
    x1, x2, y1, y2 = model_selection.train_test_split(train, target, test_size=0.2)#, random_state=25)
    num_round = 10000
    model = lgb.train(param,
                      lgb.Dataset(x1, y1), #categorical_feature = categorical_features),
                      num_round,
                      lgb.Dataset(x2, y2), #categorical_feature = categorical_features),
                      verbose_eval=100,
                      early_stopping_rounds=200)
    auc.append(model.best_score['valid_0']['auc'])
print(auc)    
#best 120,40 .7365
# 120,60 .7354
# 120,100 .7349
# 100, 100 .7343
# 100,60 .7340
# 120,30 .7337

/Users/Enzo.Calogero/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.715047
[200]	valid_0's auc: 0.721757
[300]	valid_0's auc: 0.724848
[400]	valid_0's auc: 0.726679
[500]	valid_0's auc: 0.728258
[600]	valid_0's auc: 0.729318
[700]	valid_0's auc: 0.730324
[800]	valid_0's auc: 0.731054
[900]	valid_0's auc: 0.731577
[1000]	valid_0's auc: 0.732137
[1100]	valid_0's auc: 0.732641
[1200]	valid_0's auc: 0.733068
[1300]	valid_0's auc: 0.733359
[1400]	valid_0's auc: 0.733665
[1500]	valid_0's auc: 0.733998
[1600]	valid_0's auc: 0.734251
[1700]	valid_0's auc: 0.734408
[1800]	valid_0's auc: 0.734653
[1900]	valid_0's auc: 0.734839
[2000]	valid_0's auc: 0.73494
[2100]	valid_0's auc: 0.735156
[2200]	valid_0's auc: 0.73531
[2300]	valid_0's auc: 0.735437
[2400]	valid_0's auc: 0.735559
[2500]	valid_0's auc: 0.735701
[2600]	valid_0's auc: 0.735828
[2700]	valid_0's auc: 0.735922
[2800]	valid_0's auc: 0.736014
[2900]	valid_0's auc: 0.73609
[3000]	valid_0's auc: 0.736172
[3100]	valid_0's au

In [ ]:
Previous used
param = {'num_leaves': 120,
         'min_data_in_leaf': 40, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.05,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         
         "verbosity": -1}

In [ ]:
#old best...
param = {'num_leaves': 120,
         'min_data_in_leaf': 40, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.05,
         "boosting": "gbdt",
         "feature_fraction": 0.3,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1}

In [ ]:
BEST!!!
param = {'num_leaves': 100,
         'min_data_in_leaf': 100, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.05,
         "boosting": "gbdt",
         "feature_fraction": 0.3,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1}
print(auc)
np.mean(auc)
#[0.7308381579060174, 0.7320804022520051, 0.7324145274772682, 0.7318521930897679, 0.7320724789343092]
#0.7318515519318736

#full set...
#[2500]	valid_0's auc: 0.736881
#[0.736880901504413]

#[2458]	valid_0's auc: 0.734311
#[0.7343107459475552]

<a id="5"></a> <br>
## 5. Submission

In [28]:
features=train.columns

In [29]:
train.shape

(8921482, 76)

In [30]:
predictions = model.predict(test[features], num_iteration=model.best_iteration)

In [31]:
sub_df = pd.DataFrame({"MachineIdentifier": test["MachineIdentifier"].values})
sub_df["HasDetections"] = predictions
sub_df[:10]

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.461210
1,00000176ac758d54827acd545b6315a5,0.460224
2,0000019dcefc128c2d4387c1273dae1d,0.580194
3,0000055553dc51b1295785415f1a224d,0.442542
4,00000574cefffeca83ec8adf9285b2bf,0.558480
5,000007ffedd31948f08e6c16da31f6d1,0.483015
6,000008f31610018d898e5f315cdf1bd1,0.183537
7,00000a3c447250626dbcc628c9cbc460,0.114496
8,00000b6bf217ec9aef0f68d5c6705897,0.009662
9,00000b8d3776b13e93ad83676a28e4aa,0.145404


In [32]:
sub_df.to_csv("submitVersion2_14.csv", index=False)

# New heading